In [1]:
import sys
sys.path.append("../")

In [2]:
from functions.floodmodel_utils import get_basin_mask, get_river_mask, reshape_scalar_predictand, reshape_multiday_predictand
import xarray as xr
#Creating a Dask local cluster for parallel computing (making the computations later on much faster)
from dask.distributed import Client, LocalCluster


In [3]:

#HYPERPARAMETERS
days_intake_length = 90
forecast_day = 30


#LIBRARY IMPORTS
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from dask.distributed import Client, LocalCluster


#Connecting to a cluster to be able to run the code locally/on the cloud
#cluster = LocalCluster()  # n_workers=10, threads_per_worker=1,
client = Client(processes=False)


/root/miniconda3/envs/main/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [4]:
#This will tell you where you dashboard will be so you can visualize your model being run
print(client.scheduler_info()['services'])

{'dashboard': 33265}


In [ ]:
glofas_loaded = xr.open_dataset("/mnt/bucket/stuarts_files/glofas_reshaped.nc")

In [ ]:
era5_loaded = xr.open_mfdataset("/mnt/bucket/stuarts_files/Elbe/reanalysis-era5-single-levels_convective_precipitation,land_sea_mask,large_scale_precipitation,runoff,slope_of_sub_gridscale_orography,soil_type,total_column_water_vapour,volumetric_soil_water_layer_1,volumetric_soil_water_layer_2_*.nc", combine="by_coords")

In [ ]:
glofas = glofas_loaded.copy()
era5 = era5_loaded.copy()
glofas = glofas.rename({'lon' : 'longitude'})
glofas = glofas.rename({'lat': 'latitude'})


In [ ]:
glofas_loaded

In [ ]:
#Selectin Area of Interest for the datasets
elbe_basin_mask = get_basin_mask(glofas['dis24'].isel(time=0), 'Elbe')
elbe_river_mask = get_river_mask(glofas['dis24'].isel(time=0))

glofas_masked = glofas.where(elbe_basin_mask, drop=True)
glofas_river_masked = glofas_masked.where(elbe_river_mask, drop=True)

era5 = era5.interp(latitude=glofas.latitude, longitude=glofas.longitude).where(elbe_basin_mask, drop=True)
X = era5
X

In [ ]:
glofas_masked['dis24'].isel(time=0).plot()

In [ ]:
glofas_river_masked['dis24'].isel(time=0).plot()

In [ ]:
y_orig = glofas_masked['dis24']
#Making a copy because y will be transformed to represent the variation of discharge. The model will be predicting the variation of discharge, not the quantity of discharge itself
y = y_orig.copy()

#Era5 will be the predictor dataset


In [ ]:
X

In [ ]:
#Reformating it to change values between 0 and 1 (1 representing flood)
y = y > 0
y

In [ ]:
Xda, yda = reshape_scalar_predictand(X_small, y_small)